In [1]:
import psutil
import os
print("Starting ResNet Weight Steganalysis Research")
print(f"Initial memory: {psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024:.2f} MB")

Starting ResNet Weight Steganalysis Research
Initial memory: 63.51 MB


In [2]:
# 1. Load model dan ekstrak bobot
from WeightExtractor import WeightExtractor
modelSelected = 'resnet50'
print("Loading model...")
extractor = WeightExtractor(f'data/models/{modelSelected}.pth')

Loading model...
Loaded state_dict directly
Total keys in state_dict: 320


In [3]:
print(extractor)

In [5]:
# Lihat statistik layer
stats = extractor.get_layer_statistics()
print("\nLayer statistics (first 10 layers):")
for i, (name, stat) in enumerate(list(stats.items())[:10]):
    print(f"{name}: shape{stat['shape']}, elements: {stat['numel']}")


Layer statistics (first 10 layers):
conv1.weight: shapetorch.Size([64, 3, 7, 7]), elements: 9408
bn1.weight: shapetorch.Size([64]), elements: 64
bn1.bias: shapetorch.Size([64]), elements: 64
bn1.running_mean: shapetorch.Size([64]), elements: 64
bn1.running_var: shapetorch.Size([64]), elements: 64
bn1.num_batches_tracked: shapetorch.Size([]), elements: 1
layer1.0.conv1.weight: shapetorch.Size([64, 64, 1, 1]), elements: 4096
layer1.0.bn1.weight: shapetorch.Size([64]), elements: 64
layer1.0.bn1.bias: shapetorch.Size([64]), elements: 64
layer1.0.bn1.running_mean: shapetorch.Size([64]), elements: 64


In [6]:
# Opsi 1: Ekstrak semua bobot convolutional
print("\n=== Extracting all convolutional weights ===")
conv_weights, layer_names = extractor.extract_conv_weights_only()


=== Extracting all convolutional weights ===
Extracting: conv1.weight, Shape: torch.Size([64, 3, 7, 7])
Extracting: layer1.0.conv1.weight, Shape: torch.Size([64, 64, 1, 1])
Extracting: layer1.0.conv2.weight, Shape: torch.Size([64, 64, 3, 3])
Extracting: layer1.0.conv3.weight, Shape: torch.Size([256, 64, 1, 1])
Extracting: layer1.0.downsample.0.weight, Shape: torch.Size([256, 64, 1, 1])
Memory usage: 305.85 MB
Extracting: layer1.1.conv1.weight, Shape: torch.Size([64, 256, 1, 1])
Extracting: layer1.1.conv2.weight, Shape: torch.Size([64, 64, 3, 3])
Extracting: layer1.1.conv3.weight, Shape: torch.Size([256, 64, 1, 1])
Extracting: layer1.2.conv1.weight, Shape: torch.Size([64, 256, 1, 1])
Extracting: layer1.2.conv2.weight, Shape: torch.Size([64, 64, 3, 3])
Memory usage: 306.32 MB
Extracting: layer1.2.conv3.weight, Shape: torch.Size([256, 64, 1, 1])
Extracting: layer2.0.conv1.weight, Shape: torch.Size([128, 256, 1, 1])
Extracting: layer2.0.conv2.weight, Shape: torch.Size([128, 128, 3, 3])
Ex

In [7]:
if not conv_weights:
    print("No convolutional weights found! Trying alternative extraction...")
    # Fallback: ekstrak semua weights yang ada
    conv_weights = []
    for name, param in extractor.state_dict.items():
        if 'weight' in name and len(param.shape) >= 2:  # Filter kemungkinan conv weights
            weight_vec = param.cpu().numpy().flatten()
            conv_weights.append(weight_vec)
            print(f"Fallback extraction: {name}, {len(weight_vec)} weights")

In [31]:
print(f"\nExtracted {len(conv_weights)} convolutional layers weight")


Extracted 53 convolutional layers weight


In [9]:
# 3. Persiapkan data training dengan parameter yang benar
print("\n=== Preparing training data ===")

# Gunakan hanya 2-3 layer pertama untuk menghemat memory
training_weights = conv_weights[:3] if len(conv_weights) >= 3 else conv_weights
        
# Buat dataset dengan parameter yang sesuai
from Autoencoder import WeightDataset
dataset = WeightDataset(
    weight_data=training_weights,
    seq_length=32,  # Sequence length lebih kecil
    max_samples=5000  # Batasi jumlah samples coba 2000
)

if len(dataset) == 0:
    # Fallback: coba dengan seq_length yang lebih kecil
    print("No data with seq_length=128, trying with smaller seq_length...")
    dataset = WeightDataset(
        weight_data=training_weights,
        seq_length=64,  # Lebih kecil
        max_samples=2000
    )
if len(dataset) == 0:
    raise ValueError("No training data available even with smaller seq_length!")



=== Preparing training data ===
Processing weight data with max_samples: 5000
Weight array 0: 9408 weights -> 294 sequences
Weight array 1: 4096 weights -> 128 sequences
Weight array 2: 36864 weights -> 1152 sequences
Dataset created: 1574 sequences of length 32


In [10]:
# DataLoader dengan batch size sangat kecil
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)
print(f"Training samples: {len(dataset)}")
print(f"Batch size: 4")
print(f"Memory before training: {psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024:.2f} MB")

Training samples: 1574
Batch size: 4
Memory before training: 396.27 MB


In [11]:
# 4. Inisialisasi model kecil
input_dim = dataset[0][0].shape[0]
print(f"Input dimension: {input_dim}")

Input dimension: 32


In [12]:
from Autoencoder import Autoencoder
autoencoder = Autoencoder(input_dim=input_dim, encoding_dim=8)
print(f"Model parameters: {sum(p.numel() for p in autoencoder.parameters()):,}")

Model parameters: 8,936


#### Start Training

In [13]:

print("\n=== Starting Training ===")
from Training import train_with_progress, train_with_memory_monitoring, simple_train
# Pilih salah satu metode training:
trained_model = train_with_memory_monitoring(autoencoder, dataloader, epochs=20, modelSelected=modelSelected)
# trained_model = simple_train(autoencoder, dataloader, epochs=2)
# trained_model = train_with_progress(autoencoder, dataloader, epochs=20)


=== Starting Training ===
Epoch 0, Batch 0, Loss: 0.926044, Memory: 453.43 MB
Epoch 0, Batch 10, Loss: 0.991969, Memory: 453.51 MB
Epoch 0, Batch 20, Loss: 0.945334, Memory: 453.50 MB
Epoch 0, Batch 30, Loss: 0.962521, Memory: 453.50 MB
Epoch 0, Batch 40, Loss: 0.934465, Memory: 453.50 MB
Epoch 0 completed. Average Loss: 0.963880
Epoch 1, Batch 0, Loss: 0.927964, Memory: 453.71 MB
Epoch 1, Batch 10, Loss: 0.915919, Memory: 453.73 MB
Epoch 1, Batch 20, Loss: 0.836275, Memory: 453.64 MB
Epoch 1, Batch 30, Loss: 0.892575, Memory: 453.64 MB
Epoch 1, Batch 40, Loss: 0.783985, Memory: 453.64 MB
Epoch 1 completed. Average Loss: 0.861073
Epoch 2, Batch 0, Loss: 0.765603, Memory: 453.73 MB
Epoch 2, Batch 10, Loss: 0.767092, Memory: 453.73 MB
Epoch 2, Batch 20, Loss: 0.740791, Memory: 453.65 MB
Epoch 2, Batch 30, Loss: 0.769932, Memory: 453.65 MB
Epoch 2, Batch 40, Loss: 0.750337, Memory: 453.65 MB
Epoch 2 completed. Average Loss: 0.776275
Epoch 3, Batch 0, Loss: 0.815750, Memory: 453.73 MB
Epo

### # Simpan model

In [18]:
import torch
import torch.nn as nn
torch.save(trained_model.state_dict(), f'data/models/{modelSelected}_autoencoder_final.pth')
print("Model saved successfully!")

Model saved successfully!


### # Test reconstruction

In [19]:

print("\n=== Testing Reconstruction ===")
trained_model.eval()
with torch.no_grad():
    test_sample, _ = dataset[0]
    test_sample = test_sample.unsqueeze(0)
    encoded, decoded = trained_model(test_sample)
    test_loss = nn.MSELoss()(decoded, test_sample)
    print(f"Test reconstruction loss: {test_loss.item():.6f}")
        
print("Research completed successfully!")


=== Testing Reconstruction ===
Test reconstruction loss: 0.391117
Research completed successfully!
